In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [49]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train_dataset, mnist_test_dataset = mnist_dataset['train'], mnist_dataset['test']

mnist_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
mnist_validation_samples = tf.cast(mnist_validation_samples, tf.int64)

mnist_test_samples = 0.1 * mnist_info.splits['test'].num_examples
mnist_test_samples = tf.cast(mnist_validation_samples, tf.int64)

def scale(image, label):
    image= tf.cast(image, tf.float64)
    image/=255.
    
    return image, label

scaled_train_validation_data = mnist_train_dataset.map(scale)

scaled_test_data = mnist_test_dataset.map(scale)

In [50]:
BUFFER_SIZE = 10000

shuffled_train_validation_data = scaled_train_validation_data.shuffle(BUFFER_SIZE)
shuffed_test_data = scaled_test_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_validation_data.take(mnist_validation_samples)
train_data = shuffled_train_validation_data.skip(mnist_validation_samples)

BATCH_SIZE= 100

train_data = train_data.batch(BATCH_SIZE)

validation_data  = validation_data.batch(mnist_validation_samples)

test_data = scaled_test_data.batch(mnist_test_samples)


validation_input, validation_target= next(iter(validation_data))

In [51]:
input_size = 784
output_size = 10
hidden_layers = 200


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    
    tf.keras.layers.Dense(hidden_layers, activation='relu'),
    tf.keras.layers.Dense(hidden_layers, activation='tanh'),
    
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_input, validation_target), verbose=2)

Epoch 1/5
540/540 - 6s - loss: 0.2581 - accuracy: 0.9235 - val_loss: 0.1310 - val_accuracy: 0.9647
Epoch 2/5
540/540 - 3s - loss: 0.0990 - accuracy: 0.9692 - val_loss: 0.0769 - val_accuracy: 0.9778
Epoch 3/5
540/540 - 3s - loss: 0.0652 - accuracy: 0.9800 - val_loss: 0.0656 - val_accuracy: 0.9822
Epoch 4/5
540/540 - 3s - loss: 0.0490 - accuracy: 0.9843 - val_loss: 0.0541 - val_accuracy: 0.9838
Epoch 5/5
540/540 - 3s - loss: 0.0376 - accuracy: 0.9878 - val_loss: 0.0376 - val_accuracy: 0.9887


In [58]:
test_loss, test_accuracy = model.evaluate(test_data)

2/2 [==============================] - 1s 234ms/step - loss: 0.0768 - accuracy: 0.9770


In [69]:
print("test loss is {0:.2f} and test accuracy is {1:.2f}".format(test_loss, test_accuracy*100.))

test loss is 0.08 and test accuracy is 97.70
